# Date pre-processing 데이터 전처리
- 데이터 형식에 대한 처리
    - 공백 문자
        - str.strip() : 양쪽 공백 제거
        - str.lstrip() : 왼쪽 공백 제거
        - str.rstip() : 오른쪽 공백 제거
    - 데이터 타입
    - 불규칙한 대소문자
    - 불규칙한 구분기호
    - 유효하지 않은 문자
    - 불규칙한 날짜 및 시간 표기

In [1]:
# 라이브러리 불러오기
import pandas as pd
import numpy as np

## 1. 날짜 형식

### 1.1. 날짜 데이터: str 타입

In [2]:

str_date = ['2022/11/01', '2022.10.31', '2021-10-09']

In [3]:
# Series로 저장
pd.Series(str_date)

0    2022/11/01
1    2022.10.31
2    2021-10-09
dtype: object

<br>

1. str 타입을 datetime 타입으로 변환 후 pandas 저장
- `pd.to_datetime()`

In [4]:
pd.to_datetime(str_date)

DatetimeIndex(['2022-11-01', '2022-10-31', '2021-10-09'], dtype='datetime64[ns]', freq=None)

<br>

2. str 타입으로 저장 후 data type 변경   

- `Series.dtype`: dtype 확인

- `Series.astype()`: dtype 변경

In [5]:
pd.Series(str_date).astype('datetime64')

0   2022-11-01
1   2022-10-31
2   2021-10-09
dtype: datetime64[ns]

<br>

### 1.2. 날짜 데이터: `timestamp` 타입

- `timestamp`: 기준시각(1970.01.01 00:00:00 UTC)로부터 몇 초가 경과했는지 표기 =`UNIX 시간`

In [6]:
stamp_date = [1234000, 1234101, 1234202, 1234300, 123450]

pd.Series(stamp_date)

0    1234000
1    1234101
2    1234202
3    1234300
4     123450
dtype: int64

<br>

- `datetime` 타입으로 변환 후 저장
- `timestampe`의 기본 `unit=ns`(nano seconds) 나노세컨즈: 10억분의 1초
- optional parameter `unit` default: `ns`, days: `D`, seconds :`s`, milli seconds: `ms`, micro seconds: `us`

In [7]:
pd.Series(pd.to_datetime(stamp_date))

0   1970-01-01 00:00:00.001234000
1   1970-01-01 00:00:00.001234101
2   1970-01-01 00:00:00.001234202
3   1970-01-01 00:00:00.001234300
4   1970-01-01 00:00:00.000123450
dtype: datetime64[ns]

<br>

- `unit='s'`: seconds

In [8]:
pd.Series(pd.to_datetime(stamp_date, unit='s'))

0   1970-01-15 06:46:40
1   1970-01-15 06:48:21
2   1970-01-15 06:50:02
3   1970-01-15 06:51:40
4   1970-01-02 10:17:30
dtype: datetime64[ns]

<br>

- `unit='D'`: days 

In [10]:
pd.Series(pd.to_datetime(stamp_date, unit='D'))
# 2038년 문제

OutOfBoundsDatetime: cannot convert input 1234000 with the unit 'D'

<br>

- `unit='ms'`

In [11]:
pd.Series(pd.to_datetime(stamp_date, unit='ms'))

0   1970-01-01 00:20:34.000
1   1970-01-01 00:20:34.101
2   1970-01-01 00:20:34.202
3   1970-01-01 00:20:34.300
4   1970-01-01 00:02:03.450
dtype: datetime64[ns]

<br>

- `unit-='us'` : default

In [12]:
pd.Series(pd.to_datetime(stamp_date, unit='us'))

0   1970-01-01 00:00:01.234000
1   1970-01-01 00:00:01.234101
2   1970-01-01 00:00:01.234202
3   1970-01-01 00:00:01.234300
4   1970-01-01 00:00:00.123450
dtype: datetime64[ns]

## 2. label 형식 통일

- data encoding 작업에 포함

<br>

- `map()`

- dict 타입으로 encoding map을 생성해서 적용

- gender: `0`=남자, `1`=여자

In [13]:
gender_df = pd.DataFrame({'gender': [0, 0, 0, 1, 0, 1]})
gender_map = {0:'M', 1:'F'}

gender_df

,gender
0,0
1,0
2,0
3,1
4,0
5,1


<br>

- `gender_df` 변수의 `gender` column값을 `map`함수를 이용해 `0`은 `M`으로, `1`은 `F`로 변환

In [14]:
gender_df['gender'].map(gender_map)

0    M
1    M
2    M
3    F
4    M
5    F
Name: gender, dtype: object

<br>

- Python 내장함수 `replace()`도 사용 가능

In [15]:
gender_df['gender'].replace(0, "M").replace(1, "F")

0    M
1    M
2    M
3    F
4    M
5    F
Name: gender, dtype: object

In [16]:
gender_df['gender'].replace([0, 1], ['M', 'F'])

0    M
1    M
2    M
3    F
4    M
5    F
Name: gender, dtype: object

In [17]:
gender_df['gender'].replace({0:'M', 1:'F'})

0    M
1    M
2    M
3    F
4    M
5    F
Name: gender, dtype: object

<br>

## 3. 문자 형식(대소문자, 기호 등) 통일

In [18]:
my_data = {'Name':['Jane', 'Albert', 'John'],
          'Age':[18, 19, 21]}

my_df = pd.DataFrame(my_data)
my_df

,Name,Age
0,Jane,18
1,Albert,19
2,John,21


<br>

- column name을 소문자로 바꾸는 방법: `str.lower()`

- 빈 list에 소문자로 변경한 column을 모두 적재한 뒤 대입

In [19]:
my_df1 = my_df.copy()

new_cols = list()

for col in my_df.columns:
    print(col.lower())
    new_cols.append(col.lower())

name
age


In [20]:
new_cols

['name', 'age']

In [21]:
my_df1.columns = new_cols

my_df1

,name,age
0,Jane,18
1,Albert,19
2,John,21


<br>

- `df.columns.str.upper()`

In [22]:
my_df2 = my_df.copy()

my_df2.columns = my_df2.columns.str.upper()
my_df2

,NAME,AGE
0,Jane,18
1,Albert,19
2,John,21


<br>

- 내부 item ('Name' column)의 모든 data를 소문자로 통일
- column `age`: 정수 자료 > exclude
- `.apply([함수명])`: 해당 함수의 return value로 column 내부 data를 일괄적으로 교체

In [23]:
def change_lower(value):
    return value.lower()

In [24]:
my_df2['NAME'].apply(change_lower)

0      jane
1    albert
2      john
Name: NAME, dtype: object

<br>

- `.apply()`는 `map`으로 대체 가능

In [25]:
print(my_df2)

my_df2['NAME'].map(change_lower)

     NAME  AGE
0    Jane   18
1  Albert   19
2    John   21


0      jane
1    albert
2      john
Name: NAME, dtype: object

<br>

- `AGE` column의 value가 20 이상이면 "성인", 19 이하면 "미성년자" return하는 `is_adult()`함수 정의 후 `.apply()`로 적용

In [26]:
def is_adult(value) -> str :
    if value >= 20:
        return "성인"
    else:
        return "미성년자"    

In [27]:
my_df2['AGE'].map(is_adult)

0    미성년자
1    미성년자
2      성인
Name: AGE, dtype: object

## 4. data value에 대한 처리

- 결측값(NaN)

- 이상치(예측 범위 밖의 value)

- 단순 중복 data

- 동일한 의미, 다른 명칭의 중복 data

- 중복 속성
    
    - 다중공선성: 통계학의 회귀분석에서 독립변수들 간에 강한 상관관계가 나타나는 문제

- 불규칙한 data 수집(differs in step, unit)

<br>

- data accumulation

In [28]:
this_sample = pd.read_csv('data/csv_exam_nan.csv')

this_sample

,math,english,science
0,70.0,NaN,NaN
1,75.0,65.0,80.0
2,NaN,NaN,NaN
3,56.0,89.0,NaN
4,89.0,95.0,83.0
5,90.0,100.0,89.0


### 4.1. 결측치 처리 - 삭제, 선택

1. 결측치`NaN`가 하나라도 있는 레코드 삭제

2. 모든 값이 결측인 레코드 삭제

3. 결측치가 하나라도 있는 데이터만 선택

<br>

1. 결측치가 하나 이상인 레코드(row) 삭제

- `df.dropna(how='any')`

- `df.dropna()` parameter default: `how='any'` > 하나라도 `NaN`이면 row 삭제

- optional parameter: `inplace=True or False` > 원본 data에 반영 여부 결정

In [39]:
this_sample.dropna()

,math,english,science
1,75.0,65.0,80.0
4,89.0,95.0,83.0
5,90.0,100.0,89.0


<br>

2. 모든 값이 `NaN`인 record만 삭제: `how='all'`

In [40]:
this_sample.dropna(how='all')

,math,english,science
0,70.0,NaN,NaN
1,75.0,65.0,80.0
3,56.0,89.0,NaN
4,89.0,95.0,83.0
5,90.0,100.0,89.0


<br>

3.결측치가 하나라도 있는 data만 선택: `df[df.isnull().any(axis=1)]` > 조건색인 활용

- `df.isnull()`: `NaN`인 cell은 `True`, 아닌 셀은 `False`로 DataFrame 출력

In [29]:
this_sample.isnull()

,math,english,science
0,False,True,True
1,False,False,False
2,True,True,True
3,False,False,True
4,False,False,False
5,False,False,False


<br>

- `df.any(axis=0)`: 해당 column에 `isnull()`의 결과값이 `True`인 셀이 있는지 체크

- default: `axis=0`: column 단위 기준

- `axis=1`: row 단위 기준

In [33]:
this_sample.isnull().any()

math       True
english    True
science    True
dtype: bool

In [34]:
this_sample.isnull().any(axis=1)

0     True
1    False
2     True
3     True
4    False
5    False
dtype: bool

In [41]:
this_sample[this_sample.isnull().any(axis=1)]

,math,english,science
0,70.0,NaN,NaN
2,NaN,NaN,NaN
3,56.0,89.0,NaN
